In [1]:
# Install required packages and set up Chrome WebDriver
!pip install selenium beautifulsoup4
!apt-get update
!apt-get -q -y install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver_manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,247 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,136 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/pp

In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
def scrape_reviews(url, driver):
    driver.get(url)
    time.sleep(2)

    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'btn-group-btn') and contains(@class, 'next')]"))
        )
        next_button.click()
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException):
        return None

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    review_elements = soup.find_all("div", class_=lambda x: x and x.startswith("topic-reply"))

    return review_elements

In [5]:
# Initialize Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # To run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

# Initialize empty lists to store the data
names = []
comments = []
dates = []

page = 1
stop = False  # Initialize the 'stop' variable

while not stop:
    url = f"https://www.pistonheads.com/gassing/topic.asp?h=0&f=23&t=1857152&i={page * 20}"

    driver.get(url)
    time.sleep(2)

    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'btn-group-btn') and contains(@class, 'next')]"))
        )
        next_button.click()
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException):
        stop = True  # Set 'stop' to True to exit the loop
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    review_elements = soup.find_all("div", class_=lambda x: x and x.startswith("topic-reply"))

    if len(review_elements) == 0:
        stop = True
        break

    for review in review_elements:
        name_element = review.find("p", class_="author js-author")
        comment_element = review.find("div", class_="phml msg-body")
        date_element = review.find("span", class_="timestamp")

        name = name_element.text.strip() if name_element else ""
        comment = comment_element.text.strip() if comment_element else ""
        date = date_element.text.strip() if date_element else ""

        names.append(name)
        comments.append(comment)
        dates.append(date)

    page += 1

In [6]:
# Close the webdriver
driver.quit()

In [7]:
# Create a pandas DataFrame from the scraped data
mini2 = pd.DataFrame({
    "Name": names,
    "Comment": comments,
    "Date": dates
})

In [8]:
# Display the review data
print(mini2)

             Name                                            Comment  \
0      cerb4.5lee  FA57REN said:\n\nspikyone said:\n This might w...   
1    RobDickinson  Brother had one of the last old mini that only...   
2      mikeswagon  TomTVR500 said:\n Forgetting for a second that...   
3      Filibuster  What is peoples view on the new options policy...   
4      Evanivitch  TomTVR500 said:\n My biggest problem with this...   
..            ...                                                ...   
117        AlexIT  Deep Thought said:\n Ah.  Right.  Thats what i...   
118        AlexIT  It did work out as you suggested Deep Thought....   
119          danp  AlexIT said:\n It did work out as you suggeste...   
120       Gaspode  AlexIT said:\n I have to sell it on the looks ...   
121        AlexIT  Gaspode said:\n Mrs Gaspode was very attached ...   

                           Date  
0     Friday 28th February 2020  
1     Friday 28th February 2020  
2     Friday 28th February 2020  

In [10]:
# Save the DataFrame to a CSV file
csv_filename = "mini2.csv"  # You can choose the desired file name
mini2.to_csv(csv_filename, index=False)

In [11]:
# Display a message after saving
print(f"Scraped data saved to {csv_filename}")

Scraped data saved to mini2.csv


In [12]:
import os

In [13]:
# Define the folder name
folder_name = "Colab_Data"

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
# Create the folder in Google Drive
drive_path = "/content/drive/MyDrive/"
folder_path = os.path.join(drive_path, folder_name)

In [16]:
# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

print(f"Folder '{folder_name}' created in Google Drive")

Folder 'Colab_Data' created in Google Drive


In [17]:
csv_filename = os.path.join(folder_path, "mini2.csv")
mini2.to_csv(csv_filename, index=False)

print(f"CSV file saved to '{csv_filename}'")

CSV file saved to '/content/drive/MyDrive/Colab_Data/mini2.csv'
